# K-Nearest Neighbors Classification Algorithim:  Python Class Implementation


In [1]:
# import libraries
import requests
from contextlib import closing
import csv
import pandas as pd
from random import seed
from scipy.spatial.distance import pdist
import numpy as np
from random import randrange
import matplotlib.pyplot as plt
from IPython.core.debugger import set_trace



In [2]:
file_name = "IRIS.csv"
cols = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'class']
dataframe = pd.read_csv(file_name, names=cols, skiprows=1)
dataset = pd.read_csv(file_name, names=cols, skiprows=1).drop('class', axis=1)
print(dataframe.head())
print(dataset.head())


   sepal_length  sepal_width  petal_length  petal_width        class
0           5.1          3.5           1.4          0.2  Iris-setosa
1           4.9          3.0           1.4          0.2  Iris-setosa
2           4.7          3.2           1.3          0.2  Iris-setosa
3           4.6          3.1           1.5          0.2  Iris-setosa
4           5.0          3.6           1.4          0.2  Iris-setosa
   sepal_length  sepal_width  petal_length  petal_width
0           5.1          3.5           1.4          0.2
1           4.9          3.0           1.4          0.2
2           4.7          3.2           1.3          0.2
3           4.6          3.1           1.5          0.2
4           5.0          3.6           1.4          0.2


In [3]:
X = dataset
y = dataframe["class"]

In [4]:
dataframe.describe()


,sepal_length,sepal_width,petal_length,petal_width
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.054000,3.758667,1.198667
std,0.828066,0.433594,1.764420,0.763161
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


In [5]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   class         150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [6]:
dataframe['class'].unique()

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

In [10]:
class KNN:

    def __init__(self, X, y):
        self.random_state = 42
        self.dataframe = dataframe  # technical debt 
        self.X = dataset
        self.y = dataframe['class']
        self.minmax = []
        self.class_val_labels = []
        self.class_uniques = []
        self.X_train = []
        self.X_test = []
        self.distance = 0.0
        self.n_neighbors = 3


    # Data Wrangling
    def string_to_float(self, X):
        """
        convert string column values to floats;
        not needed -> values are floats   
        """
        pass


    def ints_to_floats(self, X):
        """
        convert integers values to floats;
        not needed -> values are floats  
        """
        pass
    
    
    def create_unique_class_id(self, dataframe):
        """
        identify unique classification values; 
        encode column values as integer labels while 
        keeping human readable format too
        """
        unique = ('Iris-setosa', 'Iris-versicolor', 'Iris-virginica')
        cat = pd.Categorical(['1', '2', '3'], categories=[unique])
        self.class_val_labels, self.class_uniques = pd.factorize(cat)

   
    # Pre-Processing
    def train_test_split(self, X):
        # set_trace()
        X_train_sz = 0.33 * len(self.X)
        copy_X = self.X
        while len(self.X_train) < X_train_sz:
          # determine values of dataset within random range
          row = randrange(len(copy_X))
          #find the value of the duplicate dataset, 
          # move the value to the end of the X_train list
          copy_XX = copy_X.pop(row)
          print(copy_XX)
          # self.X_train.append(copy_X.pop(row))
        # remainder of split assigned to X_test
        self.X_test = copy_X


    def min_max(self):
        """
        find the minimum and maximum values for each column;
        create a list of these values
        """       
        for row in self.X_train:            
            # for each row in the dataset, find the minimum value
            minimum = min(row) # row[i].min(self.X_train)   
            # for each row in the dataset, find the maximum value
            maximum = min(row) # row[i].max(self.X_train)  

            # add both minimum and maximum values to 
            # the end of the minmax list 
            self.minmax.append([minimum, maximum])  


    def normalize(self):
        """
        rescale dataset columns to range 0-1;
        must be called after train/test split to avoid leakage/bias;
        returns ndarry of shape(n_samples, n_features) transformed data
        """
        for row in self.X_train:
          for i in range(len(row)):
            row[i] = (row[i] - self.minmax[i][0]) / (self.minmax[i][1] - self.minmax[i][0])     
    

    def calculate_ed(self, row1, row2):
        """
        calculate euclidean distance between vectors
        """
        # convert to numpy arrays (just in case)
        row1, row2 = np.array(row1), np.array(row2)

        # iterate over each dimension of the vectors
        for i in range(len(row1) - 1):
            # increment distance
            self.distance += (row1[i] - row2[i]) **2

        # return the square root
        return np.sqrt(self.distance)


    # def cross_val_split(self):
    #     """
    #     split dataset into k folds
    #     """
    #     # create empty list for split dataset and copy dataset to list
    #     self.split_X_test = []
    #     self.copy_X_test = list(self.X_test)

    #     # determine fold size
    #     fold_sz = int(len(self.X_test) / self.k_folds)

    #     for _ in range(self.k_folds):
    #       # create empty empty list to hold folds
    #       self.fold = []
    #       while len(self.folds) < fold:
    #         # determine index values of dataset within random range
    #         idx = randrange(len(copy_X_test))
    #         # at an index value of the duplicate dataset list, 
    #         # move the value to the end of the fold list
    #         self.fold.append(copy_X_test.pop(idx))
    #       # append the split_dataset with the fold list
    #       self.split_X_test.append(self.fold)
    #     self.X_test_folded = self.split_X_test

    
    # Implement K-NN Algorithm
    def get_neighbors(self, X_train, test_row, num_neighbors):
        """
        locate most similar neighbors
        """
        dist = []

        ## TODO...complete in full OOP
        
    

    def predict_class(self):
        """
        make a prediction with neighbors
        """
        pass
        ## TODO...complete in full OOP


    def k_nn(self):
        """
        K-NN algorithm
        """
        pass
        ## TODO...complete in full OOP

    
    # Evaluate K-NN Algorithm
    def accuracy_metrics(self):
        """
        calculate accuracy (prediction error) percentage;
        the set of labels predicted for a sample must 
        exactly match the corresponding set of labels 
        in y_true
        """
        correct = 0
        for i in range(len(actual)):
          if actual[i] == predicted[i]:
            correct +=1
            return correct / float(len(actual)) * 100

        ## TODO...refactor from psuedo to OOP


    # def cross_val_metric(self):
    #     """
    #     evaluate algorithm (model rules) using cross validation; 
    #     validated on the remaining part of the data 
    #     (i.e., test set) to compute a performance measure such 
    #     as accuracy
    #     """
    #     folds = cross_val_split(X_test, k_folds)
    #     self.scores = []

    #     # TODO...refactor in OOP

    #     for f in folds:
    #       train_set = list(folds)
    #       train_set.remove(f)
    #       train_set = sum(train_set, [])

    #       test_set = []
    #       for row in fold:
    #         copy_row = list(row)
    #         test_set.append(copy_row)
    #         # assumes the last value is output
    #         row_copy[-1] = None


if __name__ == '__main__':
    # X = dataset
    # y = dataframe["class"]

    # Test KNN class initializtion
    knn = KNN(X, y)
    print(X.head())
    print(X.tail())
    X.info()
    print(y.head())

    # test create_unigue_class_id method
    knn.create_unique_class_id(dataframe)
    print(knn.class_val_labels)

    # test train_test_split method
    knn.train_test_split(X)
    print(knn.X_test.head())

    # test min_max method
    knn.min_max(X_train)
    print(knn.minmax)

    # test nortmalize method
    # knn.normalize(X, minmax)
    # print()

    # test get_neighbors method

    # test predict_class method

    # test k_nn method

    # test accuracy_metrics method

     

   sepal_length  sepal_width  petal_length  petal_width
0           5.1          3.5           1.4          0.2
1           4.9          3.0           1.4          0.2
2           4.7          3.2           1.3          0.2
3           4.6          3.1           1.5          0.2
4           5.0          3.6           1.4          0.2
     sepal_length  sepal_width  petal_length  petal_width
145           6.7          3.0           5.2          2.3
146           6.3          2.5           5.0          1.9
147           6.5          3.0           5.2          2.0
148           6.2          3.4           5.4          2.3
149           5.9          3.0           5.1          1.8
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float

AttributeError: 'KNN' object has no attribute 'train_test_split'

# K-Nearest Neighbors Classification Algorithim:  SciKit-Learn Model Implementation

In [11]:
# import packages
from sklearn.datasets import load_iris
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import NeighborhoodComponentsAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score


In [12]:
n_neighbors = 3
random_state = 42

# load dataset
X = dataset
y = dataframe["class"]

In [13]:
# split into train/test set
X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.67, random_state=42, stratify = y)

In [14]:
# print(X_train)
# print(X_test)
# print(y_train)
# print(y_test)

In [15]:
# reduce dimensions to 2 with neighborhood component analysis
nca = make_pipeline(
    MinMaxScaler(),
    NeighborhoodComponentsAnalysis(n_components=2, 
                           random_state=random_state)
    )


In [16]:
# use nearest neighbor classifier to evaluate  the methoda
knn =  KNeighborsClassifier(n_neighbors=n_neighbors)

# fit the method's model
nca.fit(X_train, y_train)

# fit nearest neighbor classifier on training set
knn.fit(nca.transform(X_train), y_train)

# calculate the nearest neighbor accuracy on the test set
knn_accuracy_sklearn = knn.score(nca.transform(X_test), y_test)

# calculate the nearest neighbor cross validation score on test set
scores = cross_val_score(knn, X_train, y_train, cv=5)


In [17]:
print(f'Accuracy Score: {knn_accuracy_sklearn}')
print(f'Cross-validation Score: {np.mean(scores)}')
# 0.967 is a good score for this dataset

Accuracy Score: 0.9702970297029703
Cross-validation Score: 0.9800000000000001
